<a href="https://colab.research.google.com/github/GSukr/FashionMNIST_DCGAN_Keras/blob/GSukr-patch-1/DCGAN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook creates a Deep convolutional generative adversarial network for FashionMNIST data with Keras library. It includes the following steps:
1. Import necessary libraries.
2. Define Generator architecture.
3. Define Discriminator architecture.
4. Compile DGGAN model.
5. Train Generator and Discriminator with real images and fake ones drawn from normal distribution. 
6. Save generated images.




Note: We first built the DCGAN architecture similar to https://colab.research.google.com/github/sakethkaparthi/Apparel-GAN/blob/master/Fashion_MNIST_GAN.ipynb#scrollTo=hz1JhdX7EL5a. However, that DCGAN produced only the images of shirts. We added batch normalization to Discriminator's network to address this issue. 

### Install required packages

It is recommended to install the following packages: keras==2.1.2, keras_adversarial.
Examples on how to install these dependencies are below:

In [1]:
!pip install --force-reinstall keras==2.1.2 #install keras compatible with keras_adversarial

  Using cached https://files.pythonhosted.org/packages/68/89/58ee5f56a9c26957d97217db41780ebedca3154392cb903c3f8a08a52208/Keras-2.1.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/72/4c/5f81e7264b0a7a8bd570810f48cd346ba36faedbd2ba255c873ad556de76/scipy-1.3.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.2 which is incompatible.
ERROR: jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.16 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR:

In [2]:
!git clone https://github.com/bstriner/keras_adversarial.git #clone keras_adversarial

fatal: destination path 'keras_adversarial' already exists and is not an empty directory.


In [3]:
!cd keras_adversarial && python setup.py install #install keras_adversarial
#you may restart the runtime 

running install
running bdist_egg
running egg_info
writing keras_adversarial.egg-info/PKG-INFO
writing dependency_links to keras_adversarial.egg-info/dependency_links.txt
writing requirements to keras_adversarial.egg-info/requires.txt
writing top-level names to keras_adversarial.egg-info/top_level.txt
writing manifest file 'keras_adversarial.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversarial/legacy.py -> build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversarial/adversarial_optimizers.py -> build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversarial/image_grid_callback.py -> build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversarial/__init__.py -> build/bdist.linux-x86_64/egg/keras_adversarial
copying build/lib/keras_adversar

### Import packages:

In [4]:
import pandas as pd # for data handling
import numpy as np # for linear algebra
import matplotlib.pyplot as plt #for visualisation
import keras #for neural networks

Using TensorFlow backend.


In [0]:
#for neural networks
from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import normal_latent_sampling, AdversarialOptimizerSimultaneous
from keras_adversarial.legacy import fit
import keras.backend as K
from keras.layers import Conv2D, Flatten, Activation, Dense, UpSampling2D, Reshape, BatchNormalization, AveragePooling2D
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.initializers import TruncatedNormal
from keras.models import Sequential
from keras_adversarial.image_grid_callback import ImageGridCallback

### Prepare data

In [6]:
# get fashion mnist data
(x_train,y_train), (x_test,y_test) = keras.datasets.fashion_mnist.load_data()

# show shapes of tensors
print("x_train shape:", x_train.shape, ", y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, ", y_test shape:", y_test.shape)

# get number of classes
nClasses = len(np.unique(y_train)) # number of output classes
print("Number of classes: ", nClasses)

# normalize grayscale pixel values (0-255) to (0,1)
x_train = x_train.astype('float32')/255 # normalized training inputs
x_test = x_test.astype('float32')/255 # normalized test inputs

# show shapes of re-shaped tensors
print("x_train shape:", x_train.shape, ", y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape, ", y_test shape:", y_test.shape)

4423680/4422102 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28) , y_train shape: (60000,)
x_test shape: (10000, 28, 28) , y_test shape: (10000,)
Number of classes:  10
x_train shape: (60000, 28, 28) , y_train shape: (60000,)
x_test shape: (10000, 28, 28) , y_test shape: (10000,)


In [0]:
latent_dim = 100 #dimention of the output 

### Build generator model: Dense-Conv1-Conv2-Conv3

In [0]:
def model_generator():
    model_g = keras.Sequential([
        #first fully connected
        Dense(3136,  input_shape=(100,), kernel_initializer=TruncatedNormal(stddev=0.02), bias_initializer=TruncatedNormal(stddev=0.02)),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        Reshape([56, 56, 1]),

        #convolutional layer 1
        Conv2D(50, kernel_size = (3,3), strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02),bias_initializer=TruncatedNormal(stddev=0.02),  padding="same"),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        UpSampling2D(size=(2, 2)),

        #convolutional layer 2
        Conv2D(25, kernel_size = (3,3), strides=(2, 2), kernel_initializer=TruncatedNormal(stddev=0.02),bias_initializer=TruncatedNormal(stddev=0.02),  padding="same"),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        UpSampling2D(size=(2, 2)),

        #convolutional layer 3
        Conv2D(1, kernel_size=(1,1), strides=(2, 2), padding="same", kernel_initializer=TruncatedNormal(stddev=0.02), bias_initializer=TruncatedNormal(stddev=0.02), activation ="sigmoid")],
        name="generator")
    return model_g



### Build discriminator model with batch normalization: Conv1-Conv2-Dense1-Dense2

In [0]:
def model_discriminator():
    model_d = keras.Sequential([
        #convolutional layer 1
        Conv2D(32, kernel_size = (5,5), strides=(1, 1), kernel_initializer=TruncatedNormal(stddev=0.02), padding="same", input_shape=(28, 28, 1)),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
        
        #convolutional layer 2
        Conv2D(64, kernel_size = (5,5), strides=(1, 1), kernel_initializer=TruncatedNormal(stddev=0.02), padding="same"),
        BatchNormalization(epsilon=1e-5),
        Activation('relu'),
        AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
        
        #first fully connected
        Flatten(),
        Dense(1024,  kernel_initializer=TruncatedNormal(stddev=0.02)),
        Activation('relu'),
        
        #second fully connected
        Dense(1, kernel_initializer=TruncatedNormal(stddev=0.02)),
        Activation('sigmoid')],
        name="discriminator")
    return model_d




In [0]:
def generator_sampler(latent_dim, generator): #data sampling for generator inputs
    def fun():
        zsamples = np.random.normal(0, 1, size=(10 * 10, latent_dim)) #sample data from normal distribution with mean 0 and standard deviation 1
        gen = dim_ordering_unfix(generator.predict(zsamples))
        return gen.reshape((10, 10, 28, 28))

    return fun

In [11]:
# generator (z -> x)
generator = model_generator()
# discriminator (x -> y)
discriminator = model_discriminator()
gan = simple_gan(generator, discriminator, normal_latent_sampling((latent_dim,)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
#NNs' architecture
generator.summary()
discriminator.summary()
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3136)              316736    
_________________________________________________________________
batch_normalization_1 (Batch (None, 3136)              12544     
_________________________________________________________________
activation_1 (Activation)    (None, 3136)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 50)        500       
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 50)        200       
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 50)        0         
__________

In [0]:
model = AdversarialModel(base_model=gan, #build the model
                             player_params=[generator.trainable_weights, discriminator.trainable_weights],
                             player_names=["generator", "discriminator"])
model.adversarial_compile(adversarial_optimizer=AdversarialOptimizerSimultaneous(), #compile the model
                              player_optimizers=[Adam(0.0003, decay=1e-4), Adam(0.0003, decay=1e-4)],
                              loss='binary_crossentropy')

In [0]:
generator_cb = ImageGridCallback("output/gan_convolutional/epoch-{:03d}.png",
                                     generator_sampler(latent_dim, generator)) #save the output after each epoch

In [0]:
#for the image dimension ordering convention  
def dim_ordering_fix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 2, 3, 1))
    
def dim_ordering_unfix(x):
    if K.image_dim_ordering() == 'th':
        return x
    else:
        return np.transpose(x, (0, 3, 1, 2))
      
def dim_ordering_shape(input_shape):
    if K.image_dim_ordering() == 'th':
        return input_shape
    else:
        return (input_shape[1], input_shape[2], input_shape[0])

In [0]:
xtrain = dim_ordering_fix(x_train.reshape((-1, 1, 28, 28)))
xtest = dim_ordering_fix(x_test.reshape((-1, 1, 28, 28)))
y = gan_targets(xtrain.shape[0])

In [18]:
%time
ytest = gan_targets(xtest.shape[0])
history = model.fit(x=xtrain, y=y, validation_data=(xtest, ytest), callbacks=[generator_cb], nb_epoch=100,
                        batch_size=100) #training the DCGAN
df = pd.DataFrame(history.history) #convert to dataframe
df.to_csv("output/gan_convolutional/history.csv") #save the loss history 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
  200/60000 [..............................] - ETA: 40s - loss: 21.6256 - generator_loss: 21.6254 - generator_yfake_loss: 10.4386 - generator_yreal_loss: 11.1868 - discriminator_loss: 1.5877e-04 - discriminator_yfake_loss: 1.0495e-04 - discriminator_yreal_loss: 5.3824e-05

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


60000/60000 [==============================] - 33s 550us/step - loss: 12.7336 - generator_loss: 12.6228 - generator_yfake_loss: 6.0371 - generator_yreal_loss: 6.5857 - discriminator_loss: 0.1108 - discriminator_yfake_loss: 0.0550 - discriminator_yreal_loss: 0.0558 - val_loss: 20.8083 - val_generator_loss: 14.2734 - val_generator_yfake_loss: 0.0297 - val_generator_yreal_loss: 14.2437 - val_discriminator_loss: 6.5349 - val_discriminator_yfake_loss: 6.5345 - val_discriminator_yreal_loss: 3.9953e-04
Epoch 2/100
60000/60000 [==============================] - 33s 554us/step - loss: 9.7514 - generator_loss: 9.5715 - generator_yfake_loss: 4.5496 - generator_yreal_loss: 5.0219 - discriminator_loss: 0.1799 - discriminator_yfake_loss: 0.0860 - discriminator_yreal_loss: 0.0939 - val_loss: 8.3808 - val_generator_loss: 7.9028 - val_generator_yfake_loss: 2.4807 - val_generator_yreal_loss: 5.4221 - val_discriminator_loss: 0.4779 - val_discriminator_yfake_loss: 0.3865 - val_discriminator_yreal_loss: 0.

In [19]:
print(history.history.keys())

dict_keys(['val_loss', 'val_generator_loss', 'val_generator_yfake_loss', 'val_generator_yreal_loss', 'val_discriminator_loss', 'val_discriminator_yfake_loss', 'val_discriminator_yreal_loss', 'loss', 'generator_loss', 'generator_yfake_loss', 'generator_yreal_loss', 'discriminator_loss', 'discriminator_yfake_loss', 'discriminator_yreal_loss'])


In [0]:
#save models' weights
generator.save("output/gan_convolutional/generator.h5")
discriminator.save("output/gan_convolutional/discriminator.h5")